# Requirements and Guidelines

This notebook serves as a **requirements document** and **guideline** for implementing Bronze, Silver, and Gold layer transformations.

**Use this notebook as a reference** when building your data transformation pipelines.

## Table of Contents
1. [Architecture Overview](#architecture-overview)
2. [Bronze Layer Requirements](#bronze-layer-requirements)
3. [Silver Layer Requirements](#silver-layer-requirements)
4. [Gold Layer Requirements](#gold-layer-requirements)
5. [Best Practices](#best-practices)
6. [Checkpoints and Validation](#checkpoints-and-validation)


## Architecture Overview

### Medallion Architecture Pattern

```
┌─────────────────┐
│   Raw Data      │  CSV/Parquet files with data quality issues
│   (raw_data)    │  - Multiple date formats
└────────┬────────┘  - Missing/null values
         │           - Invalid data types
         │           - Bad records (negative, out of range)
         ▼
┌─────────────────┐
│  Bronze Layer   │  Cleaned and standardized data
│   (bronze)      │  - Standardized formats
└────────┬────────┘  - Basic data type conversions
         │           - Data quality flags
         │           - Metadata tracking
         ▼
┌─────────────────┐
│  Silver Layer   │  Transformed and enriched data
│   (silver)      │  - Business logic applied
└────────┬────────┘  - Data enrichment (joins)
         │           - Derived fields calculated
         │           - Single source of truth
         ▼
┌─────────────────┐
│   Gold Layer    │  Curated and aggregated data
│    (gold)       │  - Pre-aggregated metrics
└─────────────────┘  - Analytics-ready datasets
                     - Business KPIs
```

### Key Principles

1. **ELT Approach**: Extract → Load → Transform (not ETL)
2. **Data Preservation**: Bronze layer preserves raw data structure
3. **Incremental Processing**: Each layer builds upon the previous
4. **Delta Tables**: Use Delta format for ACID transactions and time travel
5. **Partitioning**: Partition large tables for performance


## Bronze Layer Requirements

### Purpose
The Bronze layer is the **first landing zone** for raw data. It focuses on:
- Initial data ingestion
- Basic cleaning and standardization
- Preserving raw data structure
- Adding metadata and quality flags

### Required Tasks

#### 1. Data Loading
- [ ] Load data from `raw_data` volume
- [ ] Handle multiple file formats (CSV, Parquet)
- [ ] Handle partitioned data (multiple files)
- [ ] Preserve source file information

#### 2. Date Standardization
- [ ] Handle multiple date formats:
  - `YYYY-MM-DD` (e.g., 2023-12-25)
  - `MM/DD/YYYY` (e.g., 12/25/2023)
  - `DD-MM-YYYY` (e.g., 25-12-2023)
  - `YYYYMMDD` (e.g., 20231225)
- [ ] Convert invalid dates to NULL
- [ ] Use `F.coalesce()` with multiple `F.to_date()` attempts

**Example Pattern:**
```python
df = df.withColumn(
    "order_date",
    F.coalesce(
        F.to_date(F.col("order_date"), "yyyy-MM-dd"),
        F.to_date(F.col("order_date"), "MM/dd/yyyy"),
        F.to_date(F.col("order_date"), "dd-MM-yyyy"),
        F.to_date(F.col("order_date"), "yyyyMMdd"),
        F.lit(None).cast(DateType())
    )
)
```

#### 3. Data Type Conversions
- [ ] Convert string IDs to integers (customer_id, product_id, etc.)
- [ ] Convert numeric strings to appropriate types (double, int)
- [ ] Ensure date columns are DateType
- [ ] Handle type conversion errors gracefully

#### 4. Basic Data Cleaning
- [ ] Handle negative values (set to NULL or flag)
- [ ] Handle out-of-range values (discounts > 100%, etc.)
- [ ] Replace obviously invalid values with NULL
- [ ] **DO NOT** apply business logic yet (save for Silver)

#### 5. Missing Value Handling
- [ ] Identify columns with null values
- [ ] Document null percentages
- [ ] Add quality flags for nulls
- [ ] **DO NOT** fill missing values yet (save for Silver)

#### 6. Metadata Addition
- [ ] Add `ingestion_timestamp` column
- [ ] Add `source_file` column (track where data came from)
- [ ] Add data quality flags:
  - `has_invalid_date`
  - `has_negative_quantity`
  - `has_invalid_discount`
  - `has_null_customer`
  - etc.

#### 7. Data Quality Checks
- [ ] Count records with data quality issues
- [ ] Report null percentages
- [ ] Identify bad records
- [ ] Log quality metrics

#### 8. Save to Bronze
- [ ] Save as Delta table format
- [ ] Use appropriate partitioning (e.g., by date for orders)
- [ ] Use `overwriteSchema` option for schema evolution
- [ ] Save to `/Volumes/databricks_training/delta_demo/bronze/{table_name}`

### Bronze Layer Checklist

For each table (geographies, products, customers, orders):

- [ ] Data loaded successfully
- [ ] Date columns standardized
- [ ] Data types converted correctly
- [ ] Invalid values handled (set to NULL)
- [ ] Metadata columns added
- [ ] Quality flags added
- [ ] Data quality summary generated
- [ ] Saved as Delta table in bronze volume
- [ ] Record count verified
- [ ] Schema validated

### Expected Output

**Bronze Tables:**
- `bronze/geographies` - Cleaned geography reference data
- `bronze/products` - Cleaned product catalog
- `bronze/customers` - Cleaned customer master data
- `bronze/orders` - Cleaned order transactions (partitioned by order_date)

**Key Characteristics:**
- Data is cleaned but not transformed
- Original structure preserved
- Quality issues identified and flagged
- Ready for Silver layer transformations


## Silver Layer Requirements

### Purpose
The Silver layer applies **business transformations** and creates a **single source of truth** for analytics. It focuses on:
- Business logic application
- Data enrichment through joins
- Handling missing values with business rules
- Calculating derived fields
- Creating analytics-ready datasets

### Required Tasks

#### 1. Load Bronze Data
- [ ] Load all bronze tables
- [ ] Verify record counts match bronze layer
- [ ] Check data quality flags from bronze

#### 2. Reference Tables (Geographies, Products)
- [ ] **Geographies**: Minimal transformation (already clean)
  - [ ] Ensure no null geography_ids
  - [ ] Standardize postal codes
  - [ ] Add updated_timestamp

- [ ] **Products**: Apply business rules
  - [ ] Handle missing prices: Use `cost * 1.5` if price is NULL
  - [ ] Handle missing categories: Set to "UNKNOWN"
  - [ ] Calculate `profit_margin = (price - cost) / price * 100`
  - [ ] Handle missing supplier_id: Set to 0
  - [ ] Remove invalid products (null product_id)

#### 3. Dimension Tables (Customers)
- [ ] Enrich with geography information (JOIN)
- [ ] Create `full_name = first_name + " " + last_name`
- [ ] Handle missing geography_id: Set to 0
- [ ] Handle missing geography info: Set country/state/city to "UNKNOWN"
- [ ] Handle missing registration_date: Set default date (e.g., 2020-01-01)
- [ ] Remove invalid customers (null customer_id)

#### 4. Fact Table (Orders)
- [ ] Enrich with product information (JOIN)
  - [ ] Get product_name, category, price, cost
- [ ] Enrich with customer information (JOIN)
  - [ ] Get full_name, country, state, city
- [ ] Apply business rules for missing values:
  - [ ] `quantity`: Default to 1 if NULL
  - [ ] `unit_price`: Use product price if NULL
  - [ ] `discount`: Default to 0.0 if NULL, cap at 1.0 (100%)
  - [ ] `shipping_cost`: Default to 0.0 if NULL
  - [ ] `order_date`: Use current_date() if NULL
  - [ ] `customer_id`: Set to 0 if NULL
  - [ ] `product_id`: Set to 0 if NULL

#### 5. Calculate Derived Fields (Orders)
- [ ] `line_total = quantity * unit_price`
- [ ] `discount_amount = line_total * discount`
- [ ] `net_amount = line_total - discount_amount`
- [ ] `total_amount = net_amount + shipping_cost`
- [ ] `profit = (unit_price - cost) * quantity`
- [ ] `year = year(order_date)`
- [ ] `month = month(order_date)`
- [ ] `quarter = quarter(order_date)`

#### 6. Data Validation
- [ ] Ensure all required fields are populated
- [ ] Verify calculated fields are correct
- [ ] Check for data integrity (foreign keys, etc.)
- [ ] Validate business rules are applied

#### 7. Save to Silver
- [ ] Save as Delta table format
- [ ] Use appropriate partitioning (orders by order_date)
- [ ] Use `overwriteSchema` option
- [ ] Save to `/Volumes/databricks_training/delta_demo/silver/{table_name}`

### Silver Layer Checklist

For each table:

**Geographies:**
- [ ] Loaded from bronze
- [ ] Null values handled
- [ ] Saved to silver

**Products:**
- [ ] Missing prices handled (cost * 1.5)
- [ ] Profit margin calculated
- [ ] Missing categories handled
- [ ] Saved to silver

**Customers:**
- [ ] Enriched with geography (JOIN)
- [ ] Full name created
- [ ] Missing values handled with defaults
- [ ] Saved to silver

**Orders:**
- [ ] Enriched with product (JOIN)
- [ ] Enriched with customer (JOIN)
- [ ] Missing values handled with business rules
- [ ] Derived fields calculated:
  - [ ] line_total
  - [ ] discount_amount
  - [ ] net_amount
  - [ ] total_amount
  - [ ] profit
  - [ ] year, month, quarter
- [ ] Saved to silver (partitioned by order_date)
- [ ] Total revenue verified
- [ ] Total profit verified

### Expected Output

**Silver Tables:**
- `silver/geographies` - Reference table
- `silver/products` - Products with profit_margin
- `silver/customers` - Customers enriched with geography
- `silver/orders` - Orders enriched with product and customer, all calculated fields

**Key Characteristics:**
- Business logic applied
- Data enriched through joins
- Missing values handled with business rules
- Derived fields calculated
- Single source of truth for analytics
- Ready for Gold layer aggregations


## Gold Layer Requirements

### Purpose
The Gold layer creates **curated and aggregated datasets** optimized for analytics and reporting. It focuses on:
- Pre-aggregated metrics
- Business KPIs
- Analytics-ready datasets
- Performance optimization

### Required Tasks

#### 1. Load Silver Data
- [ ] Load all silver tables
- [ ] Verify data quality
- [ ] Check record counts

#### 2. Create Aggregated Tables

**Table 1: Sales Summary by Date**
- [ ] Group by: `order_date`, `year`, `month`, `quarter`
- [ ] Aggregations:
  - [ ] `total_orders` (count of order_id)
  - [ ] `total_quantity` (sum of quantity)
  - [ ] `total_revenue` (sum of total_amount)
  - [ ] `total_discount` (sum of discount_amount)
  - [ ] `total_shipping` (sum of shipping_cost)
  - [ ] `total_profit` (sum of profit)
  - [ ] `avg_order_value` (avg of total_amount)
  - [ ] `unique_customers` (count distinct customer_id)
  - [ ] `unique_products` (count distinct product_id)
- [ ] Calculated fields:
  - [ ] `profit_margin_pct = (total_profit / total_revenue) * 100`
- [ ] Order by: `order_date`
- [ ] Save to: `gold/sales_by_date`

**Table 2: Sales Summary by Product Category**
- [ ] Group by: `category`
- [ ] Aggregations:
  - [ ] `total_orders`
  - [ ] `total_quantity_sold`
  - [ ] `total_revenue`
  - [ ] `total_profit`
  - [ ] `avg_order_value`
  - [ ] `avg_unit_price`
  - [ ] `unique_products`
  - [ ] `unique_customers`
- [ ] Calculated fields:
  - [ ] `profit_margin_pct`
  - [ ] `revenue_share_pct = (category_revenue / total_revenue) * 100`
- [ ] Order by: `total_revenue DESC`
- [ ] Save to: `gold/sales_by_category`

**Table 3: Sales Summary by Geography**
- [ ] Group by: `country`, `state`, `city`
- [ ] Aggregations:
  - [ ] `total_orders`
  - [ ] `total_quantity_sold`
  - [ ] `total_revenue`
  - [ ] `total_profit`
  - [ ] `avg_order_value`
  - [ ] `unique_customers`
  - [ ] `unique_products`
- [ ] Calculated fields:
  - [ ] `profit_margin_pct`
- [ ] Order by: `total_revenue DESC`
- [ ] Save to: `gold/sales_by_geography`

**Table 4: Customer Analytics**
- [ ] Group by: `customer_id`, `full_name`, `country`, `state`, `city`
- [ ] Aggregations:
  - [ ] `total_orders`
  - [ ] `total_items_purchased` (sum of quantity)
  - [ ] `lifetime_value` (sum of total_amount)
  - [ ] `avg_order_value`
  - [ ] `first_order_date` (min of order_date)
  - [ ] `last_order_date` (max of order_date)
  - [ ] `unique_products_purchased`
  - [ ] `unique_categories_purchased`
- [ ] Calculated fields:
  - [ ] `days_since_first_order`
  - [ ] `days_since_last_order`
  - [ ] `avg_days_between_orders`
  - [ ] `customer_segment`:
    - VIP: lifetime_value >= 10000
    - Premium: lifetime_value >= 5000
    - Regular: lifetime_value >= 1000
    - New: otherwise
- [ ] Order by: `lifetime_value DESC`
- [ ] Save to: `gold/customer_analytics`

**Table 5: Product Performance**
- [ ] Group by: `product_id`, `product_name`, `category`
- [ ] Aggregations:
  - [ ] `total_orders`
  - [ ] `total_quantity_sold`
  - [ ] `total_revenue`
  - [ ] `total_profit`
  - [ ] `avg_selling_price`
  - [ ] `min_price`
  - [ ] `max_price`
  - [ ] `unique_customers`
- [ ] Calculated fields:
  - [ ] `profit_margin_pct`
  - [ ] `avg_quantity_per_order`
  - [ ] `product_rank` (dense_rank by total_revenue)
- [ ] Order by: `total_revenue DESC`
- [ ] Save to: `gold/product_performance`

**Table 6: Monthly Sales Trend**
- [ ] Group by: `year`, `month`, `quarter`
- [ ] Aggregations:
  - [ ] `total_orders`
  - [ ] `total_quantity`
  - [ ] `total_revenue`
  - [ ] `total_profit`
  - [ ] `avg_order_value`
  - [ ] `unique_customers`
  - [ ] `unique_products`
- [ ] Calculated fields:
  - [ ] `profit_margin_pct`
  - [ ] `month_name` (January, February, etc.)
- [ ] Order by: `year`, `month`
- [ ] Save to: `gold/monthly_sales_trend`

#### 3. Data Validation
- [ ] Verify aggregation calculations
- [ ] Check for null values in key metrics
- [ ] Validate business logic (segments, ranks, etc.)
- [ ] Compare totals match silver layer

#### 4. Save to Gold
- [ ] Save all tables as Delta format
- [ ] No partitioning needed (already aggregated)
- [ ] Use `overwriteSchema` option
- [ ] Save to `/Volumes/databricks_training/delta_demo/gold/{table_name}`

### Gold Layer Checklist

- [ ] **Sales by Date**
  - [ ] All aggregations calculated
  - [ ] Profit margin calculated
  - [ ] Saved to gold

- [ ] **Sales by Category**
  - [ ] All aggregations calculated
  - [ ] Revenue share calculated
  - [ ] Saved to gold

- [ ] **Sales by Geography**
  - [ ] All aggregations calculated
  - [ ] Saved to gold

- [ ] **Customer Analytics**
  - [ ] Lifetime value calculated
  - [ ] Customer segments assigned
  - [ ] Date calculations correct
  - [ ] Saved to gold

- [ ] **Product Performance**
  - [ ] All aggregations calculated
  - [ ] Product rank calculated
  - [ ] Saved to gold

- [ ] **Monthly Sales Trend**
  - [ ] All aggregations calculated
  - [ ] Month names added
  - [ ] Saved to gold

### Expected Output

**Gold Tables:**
- `gold/sales_by_date` - Daily sales metrics
- `gold/sales_by_category` - Category performance
- `gold/sales_by_geography` - Geographic performance
- `gold/customer_analytics` - Customer insights and segments
- `gold/product_performance` - Product rankings and metrics
- `gold/monthly_sales_trend` - Monthly trends

**Key Characteristics:**
- Pre-aggregated for fast queries
- Business KPIs included
- Analytics-ready
- Optimized for reporting tools


## Best Practices

### 1. Code Organization
- Use clear variable names: `df_{table}_{layer}`
- Add comments explaining business logic
- Break complex transformations into steps
- Use intermediate variables for readability

### 2. Error Handling
- Always check for null values before operations
- Use `F.coalesce()` for safe null handling
- Validate data types before casting
- Handle division by zero (use `F.when()` conditions)

### 3. Performance Optimization
- Use appropriate partitioning (by date for time-series data)
- Avoid unnecessary shuffles
- Use broadcast joins for small reference tables
- Cache intermediate results if reused

### 4. Data Quality
- Always add quality flags
- Track data lineage (source_file, ingestion_timestamp)
- Validate record counts between layers
- Document assumptions and business rules

### 5. Delta Table Best Practices
- Always use `.format("delta")`
- Use `.mode("overwrite")` for full refresh
- Use `.option("overwriteSchema", "true")` for schema evolution
- Partition large tables appropriately
- Use `F.current_timestamp()` for audit columns

### 6. Transformation Patterns

**Pattern 1: Safe Null Handling**
```python
df = df.withColumn("column", F.coalesce(F.col("column"), F.lit(default_value)))
```

**Pattern 2: Conditional Logic**
```python
df = df.withColumn("segment",
    F.when(F.col("value") >= 10000, "VIP")
     .when(F.col("value") >= 5000, "Premium")
     .otherwise("Regular"))
```

**Pattern 3: Calculated Fields**
```python
df = df.withColumn("total", 
    F.col("quantity") * F.col("unit_price"))
```

**Pattern 4: Aggregations**
```python
df_agg = df.groupBy("category").agg(
    F.count("order_id").alias("total_orders"),
    F.sum("revenue").alias("total_revenue")
)
```

### 7. Common Pitfalls to Avoid

❌ **Don't:**
- Apply business logic in Bronze layer
- Fill missing values in Bronze (just flag them)
- Hard-code values (use variables/constants)
- Skip data quality checks
- Forget to handle nulls in calculations
- Use string concatenation for dates

✅ **Do:**
- Preserve raw data structure in Bronze
- Apply business rules in Silver
- Use `F.coalesce()` for null handling
- Add metadata and quality flags
- Validate calculations
- Use proper date functions


## Checkpoints and Validation

### After Bronze Layer

Run these checks:

```python
# 1. Verify all tables exist
tables = ["geographies", "products", "customers", "orders"]
for table in tables:
    df = spark.read.format("delta").load(f"{bronze_path}/{table}")
    print(f"{table}: {df.count():,} records")

# 2. Check data quality flags
df_orders = spark.read.format("delta").load(f"{bronze_path}/orders")
print("Data Quality Issues:")
print(f"Invalid dates: {df_orders.filter(F.col('has_invalid_date')).count():,}")
print(f"Negative quantities: {df_orders.filter(F.col('has_negative_quantity')).count():,}")

# 3. Check null percentages
for col in df_orders.columns:
    null_count = df_orders.filter(F.col(col).isNull()).count()
    null_pct = (null_count / df_orders.count()) * 100
    if null_pct > 0:
        print(f"{col}: {null_pct:.2f}% null")
```

### After Silver Layer

Run these checks:

```python
# 1. Verify all tables exist
tables = ["geographies", "products", "customers", "orders"]
for table in tables:
    df = spark.read.format("delta").load(f"{silver_path}/{table}")
    print(f"{table}: {df.count():,} records")

# 2. Verify business logic
df_orders = spark.read.format("delta").load(f"{silver_path}/orders")
print("Business Logic Validation:")
print(f"Total Revenue: ${df_orders.agg(F.sum('total_amount')).collect()[0][0]:,.2f}")
print(f"Total Profit: ${df_orders.agg(F.sum('profit')).collect()[0][0]:,.2f}")

# 3. Check for nulls in key fields
key_fields = ["customer_id", "product_id", "order_date", "total_amount"]
for field in key_fields:
    null_count = df_orders.filter(F.col(field).isNull()).count()
    print(f"{field} nulls: {null_count}")

# 4. Verify calculated fields
df_orders.select("line_total", "discount_amount", "net_amount", "total_amount").show(10)
```

### After Gold Layer

Run these checks:

```python
# 1. Verify all tables exist
gold_tables = [
    "sales_by_date",
    "sales_by_category",
    "sales_by_geography",
    "customer_analytics",
    "product_performance",
    "monthly_sales_trend"
]

for table in gold_tables:
    df = spark.read.format("delta").load(f"{gold_path}/{table}")
    print(f"{table}: {df.count():,} records")

# 2. Verify aggregations match silver
df_orders_silver = spark.read.format("delta").load(f"{silver_path}/orders")
df_sales_by_date = spark.read.format("delta").load(f"{gold_path}/sales_by_date")

silver_total = df_orders_silver.agg(F.sum("total_amount")).collect()[0][0]
gold_total = df_sales_by_date.agg(F.sum("total_revenue")).collect()[0][0]

print(f"Silver total revenue: ${silver_total:,.2f}")
print(f"Gold total revenue: ${gold_total:,.2f}")
print(f"Match: {abs(silver_total - gold_total) < 0.01}")

# 3. Sample each table
for table in gold_tables:
    df = spark.read.format("delta").load(f"{gold_path}/{table}")
    print(f"\n{table} sample:")
    df.show(5, truncate=False)
```

### Success Criteria

✅ **Bronze Layer is successful if:**
- All 4 tables created
- Data types are correct
- Dates are standardized
- Quality flags are present
- No data loss (record counts reasonable)

✅ **Silver Layer is successful if:**
- All 4 tables created
- Business logic applied correctly
- Joins successful (no unexpected nulls)
- Calculated fields are correct
- Total revenue/profit are reasonable

✅ **Gold Layer is successful if:**
- All 6 tables created
- Aggregations match silver totals
- Business metrics are calculated
- Customer segments assigned
- Product ranks calculated
- Ready for analytics


## Quick Reference

### Paths
```python
raw_data_path = "/Volumes/databricks_training/delta_demo/raw_data"
bronze_path = "/Volumes/databricks_training/delta_demo/bronze"
silver_path = "/Volumes/databricks_training/delta_demo/silver"
gold_path = "/Volumes/databricks_training/delta_demo/gold"
```

### Common Imports
```python
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window
```

### Standard Delta Write Pattern
```python
df.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .partitionBy("order_date") \  # Optional, for large tables
    .save(f"{path}/{table_name}")
```

### Standard Delta Read Pattern
```python
df = spark.read.format("delta").load(f"{path}/{table_name}")
```

### Date Standardization Pattern
```python
df = df.withColumn(
    "date_column",
    F.coalesce(
        F.to_date(F.col("date_column"), "yyyy-MM-dd"),
        F.to_date(F.col("date_column"), "MM/dd/yyyy"),
        F.to_date(F.col("date_column"), "dd-MM-yyyy"),
        F.to_date(F.col("date_column"), "yyyyMMdd"),
        F.lit(None).cast(DateType())
    )
)
```

### Null Handling Pattern
```python
# Default value
df = df.withColumn("column", F.coalesce(F.col("column"), F.lit("UNKNOWN")))

# Business rule
df = df.withColumn("price", F.coalesce(F.col("price"), F.col("cost") * 1.5))
```

### Aggregation Pattern
```python
df_agg = df.groupBy("group_column").agg(
    F.count("id").alias("count"),
    F.sum("amount").alias("total"),
    F.avg("amount").alias("average"),
    F.countDistinct("customer_id").alias("unique_customers")
)
```

---

## Table Registration for Dashboards

### Why Register Tables?

When you save data as Delta format to volumes (using `.save()`), the data is stored as **files**. To use them in:
- **Databricks SQL** queries
- **Databricks Dashboards**
- **SQL Analytics**
- **BI Tools** (Tableau, Power BI, etc.)

You **MUST register them as tables** in Unity Catalog.

### Best Practices

1. **Always Register Gold Tables** ✅
   - Gold tables are optimized for dashboards
   - Pre-aggregated = fast queries
   - Primary source for analytics

2. **Register Silver Tables** ✅
   - For ad-hoc SQL queries
   - Complex dashboards needing detailed data
   - Data exploration

3. **Optional: Bronze Tables** ⚠️
   - Only if needed for data quality monitoring
   - Usually not needed for dashboards

4. **Create Views** ✅
   - For specific dashboard queries
   - Pre-filtered data (e.g., current year)
   - Simplified schemas
   - Security (hide sensitive columns)

### How to Register Tables

**Pattern:**
```python
# Register a Delta table
spark.sql(f"""
    CREATE TABLE IF NOT EXISTS databricks_training.delta_demo.gold_{table_name}
    USING DELTA
    LOCATION '/Volumes/databricks_training/delta_demo/gold/{table_name}'
""")
```

**After Registration:**
- Tables are accessible via SQL: `SELECT * FROM databricks_training.delta_demo.gold_sales_by_date`
- Can be used in Databricks SQL and Dashboards
- Can be connected to BI tools

### When to Create Views

Create views when you need:
- **Filtered data**: `WHERE year = 2024`
- **Simplified schema**: Only selected columns
- **Combined metrics**: Joins from multiple tables
- **Security**: Hide sensitive columns

**Pattern:**
```python
spark.sql("""
    CREATE OR REPLACE VIEW databricks_training.delta_demo.v_current_year_sales AS
    SELECT * FROM databricks_training.delta_demo.gold_sales_by_date
    WHERE year = YEAR(CURRENT_DATE())
""")
```

### Checklist for Table Registration

After creating Gold layer tables:
- [ ] Register all Gold tables in Unity Catalog
- [ ] Verify tables are accessible via SQL
- [ ] Create views for common dashboard queries
- [ ] Test queries in Databricks SQL
- [ ] Document table names and purposes

**See `06_Register_Tables_for_Dashboards.ipynb` for complete implementation.**

---

## Next Steps

1. Review this requirements document
2. Run `01_Setup_Environment.ipynb` to create data
3. Implement Bronze layer transformations
4. Implement Silver layer transformations
5. Implement Gold layer aggregations
6. **Register tables for dashboards** (see `06_Register_Tables_for_Dashboards.ipynb`)
7. Use validation checkpoints to verify each layer
8. Compare your results with the reference notebooks
